In [2]:
import sys
import pprint
import json
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import pandas as pd
from nltk.translate.bleu_score import SmoothingFunction
import pandas as pd
import numpy as np
import docker
import os
import re
from matplotlib.font_manager import FontProperties
from string import punctuation
import gc
from xml.dom.minidom import parse
import xml.dom.minidom
punc = punctuation
smooth = SmoothingFunction()

In [17]:
def to_file(snippet,typeName,addr):
    for index,row in snippet.iterrows():
        data = row['code']
        file = open(os.path.join(addr, typeName, str(index) + '.sol'),'w')
        file.write(data)
        file.close()

In [4]:
#to_file将csv结果存储成sol文件格式
snippet_contract = pd.read_csv("/home/yfliu/paper_data/conFunFormat/snippet_contract.csv")
snippet_function = pd.read_csv("/home/yfliu/paper_data/conFunFormat/snippet_function.csv")
to_file(snippet_contract,'contract','/home/yfliu/paper_data/snippetSolFmt')
to_file(snippet_function,'function','/home/yfliu/paper_data/snippetSolFmt')

In [18]:
code_contract = pd.read_csv("/home/yfliu/paper_data/conFunFormat/code_contract.csv")
code_function = pd.read_csv("/home/yfliu/paper_data/conFunFormat/code_function.csv")
to_file(code_function,'function','/home/yfliu/paper_data/codeSolFmt')
to_file(code_contract,'contract','/home/yfliu/paper_data/codeSolFmt')

# xml dealer

In [22]:
contractResAddr = '/home/yfliu/paper_data/snippetSolFmt/contract_contracts-consistent-crossclones/contract_contracts-consistent-crossclones-0.00.xml'
functionResAddr = '/home/yfliu/paper_data/snippetSolFmt/function_functions-consistent-crossclones/function_functions-consistent-crossclones-0.00.xml'
conStoreAddr = '/home/yfliu/paper_data/matchRes/nicadCConMatch.pic'
funStoreAddr = '/home/yfliu/paper_data/matchRes/nicadCFunMatch.pic'

In [48]:
functionResAddr = '/home/yfliu/paper_data/snippetSolFmt/function_functions-consistent-crossclones/function_functions-consistent-crossclones-0.30.xml'
contractResAddr = '/home/yfliu/paper_data/snippetSolFmt/contract_contracts-consistent-crossclones/contract_contracts-consistent-crossclones-0.30.xml'
conStoreAddr = '/home/yfliu/paper_data/matchRes/nicad32CConMatch.pic'
funStoreAddr = '/home/yfliu/paper_data/matchRes/nicad32CFunMatch.pic'

In [11]:
functionResAddr = '/home/yfliu/paper_data/snippetSolFmt/function_functions-consistent-crossclones/function_functions-consistent-crossclones-0.50.xml'
contractResAddr = '/home/yfliu/paper_data/snippetSolFmt/contract_contracts-consistent-crossclones/contract_contracts-consistent-crossclones-0.50.xml'
conStoreAddr = '/home/yfliu/paper_data/matchRes/nicad3ConMatch_0.5.pic'
funStoreAddr = '/home/yfliu/paper_data/matchRes/nicad3FunMatch_0.5.pic'

In [14]:
functionResAddr = '/home/yfliu/paper_data/snippetSolFmt/function_functions-consistent-crossclones/function_functions-consistent-crossclones-0.80.xml'
contractResAddr = '/home/yfliu/paper_data/snippetSolFmt/contract_contracts-consistent-crossclones/contract_contracts-consistent-crossclones-0.80.xml'
conStoreAddr = '/home/yfliu/paper_data/matchRes/nicad3ConMatch_0.8.pic'
funStoreAddr = '/home/yfliu/paper_data/matchRes/nicad3FunMatch_0.8.pic'

In [12]:
def xmlDeal(addr, sniFlag):
    res = {}
    DOMTree = xml.dom.minidom.parse(addr)
    collection = DOMTree.documentElement
    pairs = collection.getElementsByTagName("clone")
    for pair in pairs:
        sources = pair.getElementsByTagName('source')
        if len(sources) > 2:
            print(sources)
        sniIdx = -1
        codeIdx = -1
        for source in sources:
            file = source.getAttribute('file')
            idx = int(file.split('.')[0].split('/')[-1])
            if sniFlag in file:
                sniIdx = idx
            else:
                codeIdx = idx
        if sniIdx in res.keys():
            res[sniIdx].append(codeIdx)
        else:
            res[sniIdx] = [codeIdx]
    return res
        

In [15]:
addr = functionResAddr
sniFlag = 'snippetSolFmt'
func = xmlDeal(addr, sniFlag)
len(func.keys())

155

In [16]:
addr = contractResAddr
sniFlag = 'snippetSolFmt'
con = xmlDeal(addr, sniFlag)
len(con.keys())

2693

In [17]:
def resDeal(data,addr):
    key = list(data.keys())
    value = list(data.values())
    dataFrame = pd.DataFrame({
        'snippet':key,
        '0':value
    })
    #return dataFrame
    dataFrame.to_pickle(addr)
f = resDeal(func,funStoreAddr)
c = resDeal(con,conStoreAddr)